In [1]:
import yaml
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna

from src.data.prepare_data import prepare_data
from src.models.utils import train_splits, imbalanced_sampling
from src.models.tuner import BayesianSearch
from src.models.model import Classifier
from src.models.feature_selection import FeatureSelection
from src.models.evaluation import Evaluation

In [2]:
# read config
with open('config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

In [3]:
# load and prepare data
df = pd.read_csv(config['data_loader']['path'])
df = prepare_data(df=df)
display(df.head())

# check class distributions
df['label'].value_counts(normalize=True)

,time,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v21,v22,v23,v24,v25,v26,v27,v28,amount,label
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


label
0    0.998273
1    0.001727
Name: proportion, dtype: float64

In [4]:
# split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:,:-1], df['label']
    , test_size=config['train_test_split']['test_size']
    , random_state=123
    , shuffle=True
    , stratify=df['label']
    )

In [5]:
# apply oversampling to the train set
X_train_rs, y_train_rs = imbalanced_sampling(
    method='over'
    , X_train=X_train
    , y_train=y_train
)

# check class distributions
print(
    y_train.value_counts(normalize=True)
    , y_train_rs.value_counts(normalize=True)
    )

label
0    0.998274
1    0.001726
Name: proportion, dtype: float64 label
0    0.5
1    0.5
Name: proportion, dtype: float64


In [6]:
# split train sets into multiple sets and check class distributions
train = train_splits(X_train_rs, y_train_rs, config['train_test_split'])

In [7]:
# set study logger
study = optuna.create_study(direction='maximize')

tuner = BayesianSearch(config=config['optimization'], algorithm='XGBClassifier')

def objective(trial) -> float:
    return tuner.fit({'X': train[1].iloc[:10000,:-1], 'y': train[1].iloc[:10000,-1]}, trial)

study.optimize(objective, n_trials=20)

print(
    f"Best trial:\n",
    f"- params: {study.best_trial.params}\n",
    f"- score: {study.best_trial.value}", "\n"*2
)

[I 2025-06-06 18:08:18,328] A new study created in memory with name: no-name-f84c0896-0773-4c88-ad52-c58594b11fcb
[I 2025-06-06 18:08:20,031] Trial 0 finished with value: 0.9975000000000002 and parameters: {'n_estimators': 67, 'max_depth': 5, 'learning_rate': 0.17034874457633475, 'gamma': 2.255044556367899, 'subsample': 0.8175959510219297}. Best is trial 0 with value: 0.9975000000000002.
[I 2025-06-06 18:08:20,815] Trial 1 finished with value: 0.9814 and parameters: {'n_estimators': 12, 'max_depth': 4, 'learning_rate': 0.35640559214085477, 'gamma': 6.176581524272886, 'subsample': 0.7925439863190766}. Best is trial 0 with value: 0.9975000000000002.
[I 2025-06-06 18:08:23,023] Trial 2 finished with value: 0.9982000000000001 and parameters: {'n_estimators': 65, 'max_depth': 18, 'learning_rate': 0.27243720666091076, 'gamma': 0.7274905816503086, 'subsample': 0.8044621073821729}. Best is trial 2 with value: 0.9982000000000001.
[I 2025-06-06 18:08:24,714] Trial 3 finished with value: 0.9897 a

Best trial:
 - params: {'n_estimators': 65, 'max_depth': 18, 'learning_rate': 0.27243720666091076, 'gamma': 0.7274905816503086, 'subsample': 0.8044621073821729}
 - score: 0.9982000000000001 




In [8]:
# greedy feature selection
clf = Classifier(
    algorithm='XGBClassifier'
    , **study.best_trial.params
    )

feature_selection = FeatureSelection(X=train[2].iloc[:,:-1], y=train[2].iloc[:,-1])
feature_selection.stats_test()
feature_selection.wrapper(clf=clf, config=config['optimization'])

feats = feature_selection.features
feats

Index(['v12', 'v14', 'v21'], dtype='object')

In [9]:
# fit best algorithm on most important features of training data 
clf = Classifier(
    algorithm='XGBClassifier'
    , **study.best_trial.params
    )
clf.fit(X=X_train_rs[feats], y=y_train_rs)

# test set evaluation
eval = Evaluation(clf=clf, threshold=0.5)
eval.fit(
    train=(X_train_rs[feats], y_train_rs),
    test=(X_test[feats], y_test)
    )

,accuracy,precision,recall,f1_score
dataset,,,,
train,0.99990,0.99980,1.00,0.99990
test,0.99902,0.73913,0.68,0.70833
